# 00 Install Dependecies

In [ ]:
!pip install selenium

In [ ]:
!pip install numpy

In [ ]:
!pip install pandas

In [ ]:
!pip install scipy

# 00 Load

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import itertools
import os
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import urllib
from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np
from pathlib import Path

In [20]:
from util_db import load_db
df = load_db(False)

# 00 Global Parameter

In [21]:
# Set general settings
XPATH = "xpath"
erwo = "*Blank*"
dash = '-'
slash = '/'
cwd = Path.cwd()
dir_main = Path.cwd().parent

In [22]:
p1 = re.compile('[0-9]{4}.[0-9]{2}.[0-9]{2}')
p2 = re.compile(r'^[A-Z0-9]+(?:-[A-Z0-9]+)*$')

rare = [
    'CR',
    'SE', 'EXSE', 'PSE',
    'GUR', 'GSE', 'GR',
    'HR','GH',
    'UR (PR)','PGR',
    'KC', 'KC+R','KC+UR',
    'M', 'M+GR', 'M+SE', 'M+SR',
    'N','R','SR',
    'UR','UL',
    'P','P+ES','P+EXSE','P+HR','P+R','PS','P+SE','P+SR','P+UR',
    '20th SE', 'QCSE',
    '10000 SE',
    'SH','H','STAR','UR (Hobby)','ST', 'MR', 'PL',
]

In [23]:
xpath_name = '//*[@id="cardname"]/h1'
xpath_pull = '//*[@id="update_list"]'
xpath_type = '//*[@id="CardTextSet"]/div[1]/div[1]/div[1]/span[2]'
xpath_level = '//*[@id="CardTextSet"]/div[1]/div[1]/div[2]/span[2]'
xpath_atk = '//*[@id="CardTextSet"]/div[1]/div[2]/div[1]/span[2]'
xpath_def = '//*[@id="CardTextSet"]/div[1]/div[2]/div[2]/span[2]'
xpath_spc = '//*[@id="CardTextSet"]/div[1]/div[3]/div/p'
xpath_year = '//*[@id="update_list"]/div[2]/div/div/div[1]'

In [24]:
# Importing Sample
db_code = np.asanyarray(df["Code"])
db_digit = np.asarray(df["Digit"])
db_name_ko = np.asarray(df["Name_KO"])
db_name_ja = np.asarray(df["Name_JA"])
db_name_en = np.asarray(df["Name_EN"])
db_type = np.asarray(df['Type'])

In [25]:
string_lengths = np.char.str_len(db_name_ko.astype('str'))
max_length = np.max(string_lengths)
print(max_length)

string_lengths = np.char.str_len(db_name_en.astype('str'))
max_length = np.max(string_lengths)
print(max_length)

string_lengths = np.char.str_len(db_name_ja.astype('str'))
max_length = np.max(string_lengths)
print(max_length)

36
56
36


# 01 Run driver in headless

In [28]:
# Create headless ChromeOptions object
options = Options()
options.add_argument('--headless')

# Create a new ChromeDriverService object with the path to the Chromedriver executable
service = Service('C://chromedriver.exe')

# Initialize Chrome driver with headless options
driver = webdriver.Chrome(service=service, options=options)

## 02 Check Running

In [29]:
def get_link (code, local):
    link = f'https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid={code}&request_locale={local}'
    driver.get(link)

In [30]:
# Sample Card
card_name = "하루 우라라"
code =  db_code[np.where(db_name_ko == card_name)][0]
local = 'en'
get_link(code, local)

# Get Name
dt_name_all = driver.find_element(By.XPATH, xpath_name)
dt_name_all=dt_name_all.text
dt_name_all.split("\n")
print(dt_name_all)

Ash Blossom & Joyous Spring


In [31]:
# Sample Card
card_name = "하루 우라라"
code =  db_code[np.where(db_name_ko == card_name)][0]
local = 'ko'
get_link(code, local)

# Get Name
dt_name_all = driver.find_element(By.XPATH, xpath_name)
dt_name_all=dt_name_all.text
dt_name_all.split("\n")
print(dt_name_all)

하루 우라라
Ash Blossom & Joyous Spring


In [32]:
# Sample Card
card_name = "하루 우라라"
code =  db_code[np.where(db_name_ko == card_name)][0]
local = 'ja'
get_link(code, local)

# Get Name
dt_name_all = driver.find_element(By.XPATH, xpath_name)
dt_name_all=dt_name_all.text
dt_name_all.split("\n")
print(dt_name_all)

はるうらら
灰流うらら
Ash Blossom & Joyous Spring


# 03 Get Release Info

In [ ]:
# Sample Card
card_name = "하루 우라라"
code =  db_code[np.where(db_name_ko == card_name)][0]
local = 'ko'
get_link(code, local)

#Get Release Info
dt_pull = driver.find_element(By.XPATH, xpath_pull)
dt_pull = dt_pull.text

dt_pull_s = dt_pull.split("\n")
dt_pull_s.pop(0)

In [ ]:
check_year=[]
for i in range(len(dt_pull_s)):
    dt_temp = dt_pull_s[i]
    gen = re.search(p1,dt_temp)
    if gen == None:
        pass
    else :
        check_year.append(i)

In [ ]:
check_pack=[]
for i in range(len(dt_pull_s)):
    dt_temp = dt_pull_s[i]
    gen = re.search(p2,dt_temp)
    if gen == None:
        pass
    else :
        check_pack.append(i)

In [ ]:
db_capsule = []
for year in check_year:
    a = dt_pull_s[year]
    try:
        b = dt_pull_s[year+1]
    except:
        b = None
    try:
        c = dt_pull_s[year+2]
    except:
        c = None
    try:
        d = dt_pull_s[year+3]
    except:
        d = None
    db_capsule.append([ a,b,c,d ])

In [ ]:
dfv = pd.DataFrame (db_capsule, columns = ["Year","Code","Pack","Rarity"])
dfv

In [ ]:
db_year, db_pack, db_rare = [], [], []

for item in check_year:
    da = dt_pull_s[item]
    try:
        db = dt_pull_s[item+1]
    except:
        db = None
    try:
        dc = dt_pull_s[item+2]
    except:
        dc = None
    try:
        dd = dt_pull_s[item+3]
    except:
        dd = None
    # Append Year
    db_year.append(da)
    # Append Pack
    if db == None:
        db = erwo
    elif re.search(p2,db):
        pass
    else:
        db = erwo
    db_pack.append(db)
    # Append Rarity
    if dd == None :
        k = "N"
    elif re.search(p1,dd):
        k = "N"
    elif  len(dd) > len(dc):
        if re.search(p1,dc):
            k = "N"
        else:
            k = dc
    else :
        if len(dd) > 8:
            k = "N"
        else:
            if re.search(p1,dc):
                k = "N"
            else:
                k = dd
    db_rare.append(k)
    # print(f"Year : {da} | Pack : {db} | Rarity : {k}")

In [ ]:
data = {"Year": db_year, "Code": db_pack, "Rarity": db_rare}
dfv =  pd.DataFrame(data)
dfv

# 04 Run for all Cards

In [55]:
code_list = np.unique(db_code).tolist()
print(len(code_list))

12175


In [69]:
dir_update_db = f'{dir_main}/data/update_code.txt'
code_list = np.loadtxt(dir_update_db, dtype='str')
code_list = [str(text) for text in code_list]
print(len(code_list))


In [72]:
local = 'ja' # en, ja, ko

nline0 = []
nline1 = []
nline2 = []
nline3 = []
nline4 = []


run_s = 0
run_e = len(code_list)

for i in range(run_s, run_e):
    code = int(code_list[i])
    get_link(code, local)

    try:
        dt_name = driver.find_element(By.XPATH, xpath_name).text
        if "\n" in dt_name:
            dt_name_sp = dt_name.split("\n")
            if local == 'ja':
                dt_name_all = [dt_name_sp[0], dt_name_sp[1]]
            elif local == 'en':
                dt_name_all = [dt_name_sp[0], '']
            elif local == 'ko':
                dt_name_all = [dt_name_sp[0], '']
            else:
                break
        else:
            if local == 'ja':
                dt_name_all = [dt_name, erwo]
            elif local == 'en':
                dt_name_all = [dt_name, '']
            elif local == 'ko':
                dt_name_all = [dt_name, '']
            else:
                break
    except:
        dt_name_all = [erwo, erwo]


    try:
        dt_type = driver.find_element(By.XPATH, xpath_type).text
    except:
        dt_type = erwo

    try:
        dt_level = driver.find_element(By.XPATH, xpath_level).text
    except:
        dt_level = erwo
    
    try:
        dt_atk = driver.find_element(By.XPATH, xpath_atk).text
    except:
        dt_atk = erwo

    try:
        dt_def = driver.find_element(By.XPATH, xpath_def).text
    except:
        dt_def = erwo

    try:
        dt_spc = driver.find_element(By.XPATH, xpath_spc).text
    except:
        dt_spc = erwo

    db_stats = [dt_type, dt_level, dt_atk, dt_def, dt_spc]

    try:
        dt_pull = driver.find_element(By.XPATH, xpath_pull)
        dt_pull = dt_pull.text
    except:
        dt_pull = erwo

    dt_pull_s = dt_pull.split("\n")
    dt_pull_s.pop(0)

    check_year = [m for m, dt_temp in enumerate(dt_pull_s) if re.search(p1, dt_temp)]

    db_year, db_pack, db_rare = [], [], []

    # fill Normal to blank
    for year in check_year:
        da = dt_pull_s[year] #1st
        try:
            db = dt_pull_s[year+1] #2nd
        except IndexError:
            db = None
        try:
            dc = dt_pull_s[year+2] #3rd
        except IndexError:
            dc = None
        try:
            dd = dt_pull_s[year+3] #4th
        except IndexError:
            dd = None

        # Append Year
        db_year.append(da)

        # Append Pack
        if re.search(p2,db):
            pass
        else:
            db = erwo
        db_pack.append(db)

        # Append Rarity
        if dd == None :
            k = "N"
        elif re.search(p1,dd):
            k = "N"
        
        elif  len(dd) > len(dc): # Override 4th to 3rd
            if re.search(p1,dc):
                k = "N"
            else:
                k = dc
        
        else :
            if len(dd) > 10: # For jp 8
                k = "N"
            else:
                if re.search(p1,dc):
                    k = "N"
                else:
                    k = dd
        db_rare.append(k)

    exp_name = []
    exp_pack = []
    exp_rare = []
    exp_stat = []
    exp_year = []

    numb2 = []
    numb2.append(str(code))
    
    exp_name.append(numb2)
    exp_name.append(dt_name_all)

    exp_pack.append(numb2)
    exp_pack.append(db_pack)

    exp_rare.append(numb2)
    exp_rare.append(db_rare)

    exp_stat.append(numb2)
    exp_stat.append(db_stats)

    exp_year.append(numb2)
    exp_year.append(db_year)

    merged0 = list(itertools.chain.from_iterable(exp_name))
    merged1 = list(itertools.chain.from_iterable(exp_pack))
    merged2 = list(itertools.chain.from_iterable(exp_rare))
    merged3 = list(itertools.chain.from_iterable(exp_stat))
    merged4 = list(itertools.chain.from_iterable(exp_year))

    data0 = "\t".join(merged0)
    data1 = "\t".join(merged1)
    data2 = "\t".join(merged2)
    data3 = "\t".join(merged3)
    data4 = "\t".join(merged4)

    # append the data
    nline0.append(data0)
    nline1.append(data1)
    nline2.append(data2)
    nline3.append(data3)
    nline4.append(data4)

    try:
        print(f'{code:05d} | {str(dt_name_all[0])[:36]:^36} | {db_year[0]} | {db_pack[0][:10]:^10} | {db_rare[0]:^7} |')

    except:
        print(f'{code:05d} | Error |')

master = f'{dir_main}/Output/'
f0 = master + 'out_nm.txt'
f1 = master + 'out_pa.txt'
f2 = master + 'out_ra.txt'
f3 = master + 'out_st.txt'
f4 = master + 'out_yr.txt'

open(f0, 'w').close()
with open(f0, 'w', encoding="utf-8") as fp:
    for item in nline0:
        fp.write("%s\n" % item)

open(f1, 'w').close()
with open(f1, 'w', encoding="utf-8") as fp:
    for item in nline1:
        fp.write("%s\n" % item)

open(f2, 'w').close()
with open(f2, 'w', encoding="utf-8") as fp:
    for item in nline2:
        fp.write("%s\n" % item)

open(f3, 'w').close()
with open(f3, 'w', encoding="utf-8") as fp:
    for item in nline3:
        fp.write("%s\n" % item)

open(f4, 'w').close()
with open(f4, 'w', encoding="utf-8") as fp:
    for item in nline4:
        fp.write("%s\n" % item)

os.startfile(f0)
os.startfile(f1)
os.startfile(f2)
os.startfile(f3)
os.startfile(f4)

04860 |                こころがわり                | 2023-02-18 | RC04-JP051 |   UR    |
05328 |                 ぞうえん                 | 2023-02-18 | RC04-JP052 |   SR    |
05496 |           ようがんまじんラヴァ・ゴーレム            | 2023-02-18 | RC04-JP001 |   SR    |
05530 |                めいすいり                 | 2023-02-18 | RC04-JP053 |   SR    |
05914 |            きょうせいだっしゅつそうち             | 2023-02-18 | RC04-JP070 |   SR    |
06845 |           サイバー・エンジェル－べんてん－           | 2023-02-18 | RC04-JP025 |   SR    |
07568 |            ローンファイア・ブロッサム             | 2023-02-18 | RC04-JP002 |   SR    |
07652 |               サモンリミッター               | 2023-02-18 | RC04-JP071 |   SR    |
08090 |          エンシェント・フェアリー・ドラゴン           | 2023-02-18 | RC04-JP031 |   UR    |
08352 |               かせきちょうさ                | 2023-02-18 | RC04-JP054 |   SR    |
08885 |               ひょうけっかい                | 2023-02-18 | RC04-JP072 |   SR    |
08933 |             エフェクト・ヴェーラー              | 2023-02-18 | RC04-JP003 |   SR    |
0919

## Run on Specified Code

In [ ]:
nline0 = []
nline1 = []
nline2 = []
nline3 = []
nline4 = []

local = 'en' # en, ja, ko

code_s = 18450
code_e = 18461

for code in range(code_s,code_e+1):
    get_link(code, local)
    try:
        dt_name = driver.find_element(By.XPATH, xpath_name).text
        if "\n" in dt_name:
            dt_name_sp = dt_name.split("\n")
            if local == 'ja':
                dt_name_all = [dt_name_sp[0], dt_name_sp[1]]
            elif local == 'en':
                dt_name_all = [dt_name_sp[0], '']
            elif local == 'ko':
                dt_name_all = [dt_name_sp[0], '']
            else:
                break
        else:
            if local == 'ja':
                dt_name_all = [dt_name, erwo]
            elif local == 'en':
                dt_name_all = [dt_name, '']
            elif local == 'ko':
                dt_name_all = [dt_name, '']
            else:
                break
    except:
        dt_name_all = [erwo, erwo]


    try:
        dt_type = driver.find_element(By.XPATH, xpath_type).text
    except:
        dt_type = erwo

    try:
        dt_level = driver.find_element(By.XPATH, xpath_level).text
    except:
        dt_level = erwo
    
    try:
        dt_atk = driver.find_element(By.XPATH, xpath_atk).text
    except:
        dt_atk = erwo

    try:
        dt_def = driver.find_element(By.XPATH, xpath_def).text
    except:
        dt_def = erwo

    try:
        dt_spc = driver.find_element(By.XPATH, xpath_spc).text
    except:
        dt_spc = erwo

    db_stats = [dt_type, dt_level, dt_atk, dt_def, dt_spc]

    try:
        dt_pull = driver.find_element(By.XPATH, xpath_pull)
        dt_pull = dt_pull.text
    except:
        dt_pull = erwo

    dt_pull_s = dt_pull.split("\n")
    dt_pull_s.pop(0)

    check_year = [m for m, dt_temp in enumerate(dt_pull_s) if re.search(p1, dt_temp)]

    db_year, db_pack, db_rare = [], [], []

    # fill Normal to blank
    for year in check_year:
        da = dt_pull_s[year] #1st
        try:
            db = dt_pull_s[year+1] #2nd
        except IndexError:
            db = None
        try:
            dc = dt_pull_s[year+2] #3rd
        except IndexError:
            dc = None
        try:
            dd = dt_pull_s[year+3] #4th
        except IndexError:
            dd = None

        # Append Year
        db_year.append(da)

        # Append Pack
        if re.search(p2,db):
            pass
        else:
            db = erwo
        db_pack.append(db)

        # Append Rarity
        if dd == None :
            k = "N"
        elif re.search(p1,dd):
            k = "N"
        
        elif  len(dd) > len(dc): # Override 4th to 3rd
            if re.search(p1,dc):
                k = "N"
            else:
                k = dc
        
        else :
            if len(dd) > 10: # For jp 8
                k = "N"
            else:
                if re.search(p1,dc):
                    k = "N"
                else:
                    k = dd
        db_rare.append(k)

    exp_name = []
    exp_pack = []
    exp_rare = []
    exp_stat = []
    exp_year = []

    numb2 = []
    numb2.append(str(code))
    
    exp_name.append(numb2)
    exp_name.append(dt_name_all)

    exp_pack.append(numb2)
    exp_pack.append(db_pack)

    exp_rare.append(numb2)
    exp_rare.append(db_rare)

    exp_stat.append(numb2)
    exp_stat.append(db_stats)

    exp_year.append(numb2)
    exp_year.append(db_year)

    merged0 = list(itertools.chain.from_iterable(exp_name))
    merged1 = list(itertools.chain.from_iterable(exp_pack))
    merged2 = list(itertools.chain.from_iterable(exp_rare))
    merged3 = list(itertools.chain.from_iterable(exp_stat))
    merged4 = list(itertools.chain.from_iterable(exp_year))

    data0 = "\t".join(merged0)
    data1 = "\t".join(merged1)
    data2 = "\t".join(merged2)
    data3 = "\t".join(merged3)
    data4 = "\t".join(merged4)

    # append the data
    nline0.append(data0)
    nline1.append(data1)
    nline2.append(data2)
    nline3.append(data3)
    nline4.append(data4)

    try:
        print(f'{code:05d} | {str(dt_name_all[0])[:36]:^36} | {db_year[0]} | {db_pack[0][:10]:^10} | {db_rare[0]:^7} |')
    except:
        print(f'{code:05d} | Error |')

master = f'{dir_main}/Output/'
f0 = master + 'out_nm.txt'
f1 = master + 'out_pa.txt'
f2 = master + 'out_ra.txt'
f3 = master + 'out_st.txt'
f4 = master + 'out_yr.txt'

open(f0, 'w').close()
with open(f0, 'w', encoding="utf-8") as fp:
    for item in nline0:
        fp.write("%s\n" % item)

open(f1, 'w').close()
with open(f1, 'w', encoding="utf-8") as fp:
    for item in nline1:
        fp.write("%s\n" % item)

open(f2, 'w').close()
with open(f2, 'w', encoding="utf-8") as fp:
    for item in nline2:
        fp.write("%s\n" % item)

open(f3, 'w').close()
with open(f3, 'w', encoding="utf-8") as fp:
    for item in nline3:
        fp.write("%s\n" % item)

open(f4, 'w').close()
with open(f4, 'w', encoding="utf-8") as fp:
    for item in nline4:
        fp.write("%s\n" % item)

os.startfile(f0)
os.startfile(f1)
os.startfile(f2)
os.startfile(f3)
os.startfile(f4)